<a href="https://colab.research.google.com/github/sahiljain01/343-a4/blob/main/HW4_Problem1_kMeans_DLPFC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COS 343 Spring 2023

**Princeton University**

**Homework #4, Problem 1**

Due 4/12/2023, 11:59pm (Princeton local time)

Please see the assignment PDF for instructions on how to submit this notebook.

# Introduction: Applying K-means on spatial transcriptomics data 

Spatial Transcriptomics (ST) is a sequencing technology that allows biologists to measure gene expression across thousands of spots on a tissue slice while simultaneously recording the two-dimensional (2D) coordinates of each spot. To conduct a spatial transcriptomics experiment, a thin tissue slice is placed on an array with a grid of spots, and the mRNA of cells within the spots is sequenced. The resulting data from a spatial transcriptomics experiment consists of a gene expression matrix, denoted by $X$, and a spatial location matrix, denoted by $Z$, where $X \in \mathbb{R}^{n \times p}$ and $Z \in \mathbb{R}^{n \times 2}$. $n$ is the number of spots on the slice and $p$ is the number of genes measured. The matrix $X$ contains the gene expression levels of each spot, where $x_{ij} \in \mathbb{R}$ is the expression level of gene $j$ in spot $i$. Each row vector $\mathbf{x}_{i \cdot}$ of $X$ represents the *expression profile* of spot $i$ and can be thought of as a feature vector of data point (spot) $i$. The matrix $Z$ contains the 2D coordinates of each spot on the tissue slice, where the $i$-th row $\mathbf{z}_{i \cdot}$ is the $x$-$y$ coordinate of spot $i$ on the tissue slice.

In this problem, we will use the K-means algorithm to cluster the spots from a spatial transcriptomics analysis of a tissue slice from the human brain, specifically the dorsolateral prefrontal cortex (DLPFC). The DLPFC is organized into distinct layers, or regions, consisting of varying types of neural cells.  The dataset we use is from [Maynard et al. (2021)](https://www.nature.com/articles/s41593-020-00787-0), where the authors manually annotated each spot in the slice as belonging to one of 6 cortical layers or the white matter, as seen in the figure in the assignment PDF. We will use K-means to try to automatically detect the 7 clusters (6 cortical layers + 1 white matter) of spots based on the $p$-dimensional feature vector (expression profile) of each spot.

# Setting up data 
First, we load all the required libraries.

The dataset contains two files: `DLPFC_spot_expression.npy` and `DLPFC_spot_location.npy`, both are numpy matrix files. The two files can be downloaded from `COS343_Spring2023_Public/Homeworks/Datasets`. Once downloaded, you need to move the two files into your own Google Drive. Then, after executing `drive.mount('/content/drive')` below, files in your Google Drive will be present in `/content/drive/MyDrive` and can be accessed by providing the path to them in your Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial import distance
import matplotlib.pyplot as plt
import seaborn as sns

import networkx as nx 
import networkx.algorithms.community as nx_comm

We now load in the gene expression matrix and the spatial location matrix. Modify the paths given to `np.load()` to the path of the two files in your own Drive.

The gene expression matrix $X$ is of size $3635 \times 50$, which means the slice has $3635$ spots, and the feature dimension of each spot is $50$. Notice that instead of the original gene expression profile of each spot, which is of dimension more than $10000$, we already did dimensionality reduction for you and reduced the dimension of the feature space to $50$.

The location matrix $Z$ is of size $3635 \times 2$, as explained above.

In [ ]:
X = np.load('/content/drive/MyDrive/PATH/TO/YOUR/DLPFC_spot_expression.npy')
print(X.shape)
Z = np.load('/content/drive/MyDrive/PATH/TO/YOUR/DLPFC_spot_location.npy')
print(Z.shape)

## 1. K-means initialization

The first step in running K-means is to choose the $k$ initial centers. Implement the following function, which given a dataset $X$ and a number $k$, randomly pick $k$ data points from $X$ as the initial centers. First, randomly permute the indices of the datapoints. Then, select the first k datapoints based on the random permutation of the indices. This allows the datapoints to be selected at random without the risk of selecting the same datapoint twice.

*Hint*: You may find the function `permutation()` in `numpy.random` useful, as documented [here](https://numpy.org/doc/stable/reference/random/generated/numpy.random.permutation.html).

In [ ]:
def kmeans_init_centers(X,k):
  """
  This function initializes k centers that are to be used on the dataset X.
  Each row of X is a single data point. k is an integer number.
  Returns k initial centers in X
  """
  X = np.copy(X)
  centers = np.zeros((k, X.shape[1]))
  #
  ######################## YOUR CODE HERE #######################################
  # Construct a random permutation of the datapoints and pick the first K items #                                              
  ###############################################################################

  ################################################################################
  #             END OF YOUR CODE                                                 #
  ################################################################################
  return centers

## 2. K-means objective

Next, we implement the objective cost of K-means, the **squared error distortion**. Given the cluster assigment of each datapoint and the center of each cluster, this function computes the squared error distortion. We give this function to you. Read to make sure you understand how to use it.

In [ ]:
def squared_error_distortion(X, labels, centers):
  """
  This function computes the distortion of a certain clustering on dataset X.

  X: a n x p matrix, where n is the number of datapoints and p is the number of features.
  labels: a n x 1 vector indicating the cluster assignment of each datapoint. Each entry is in range [0,..,k-1].
  centers: a k x p matrix, where the k-th row is the vector representing the center of cluster k.

  Returns the distortion of this clustering.
  """
  distortion = 0
  n = X.shape[0]
  for i in range(n):
    distortion += np.linalg.norm(X[i] - centers[labels[i]]) ** 2
  distortion = distortion / n
  return distortion

## 3. Running K-means

Now we run the K-means algorithm to cluster the spots in the DLPFC slice $X$ into $k=7$ clusters. 

You do not need to implement the algorithm yourself. Instead, you will use `sklearn.cluster.KMeans()`, which is already imported above for you. The documentation of the method can be found at https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html. Run the algorihtm $1000$ times on $X$, each time initialized with a different set of $k$ random centers chosen by `kmeans_init_centers()`. Record the distortions of the resulting $1000$ outcomes using `squared_error_distortion()`, and keep track of the clustering that achieves the best distortion. Record the number of times you run the Lloyd algorithm before finding the run that scored highest among your $1000$ runs.

Note: you must set `algorithm='lloyd'` and `n_init=1` when calling `KMeans()`. When `KMeans()` finishes, the `labels_` parameter stores the cluster assignment of each data point, and the `cluster_centers_` parameter stores the centers of each cluster.

In [ ]:
k = 7
distortions = []
best_cluster = None
best_distortion = float('inf')
######################## YOUR CODE HERE #######################################
# Run the algorithm 1000 times, record the distortions and the best clustering#                                              
###############################################################################

################################################################################
#             END OF YOUR CODE                                                 #
################################################################################

Construct a histogram of the squared error distortions of the resulting $1000$ outcomes with 50 bins. Documentation can be found at https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html.

In [ ]:
######################## YOUR CODE HERE #######################################
#                Draw a histogram of the 1000 distortions                     #                                              
###############################################################################

################################################################################
#             END OF YOUR CODE                                                 #
################################################################################

We provide the following code to visualize the best clustering obtained by your K-means algorithm.

In [ ]:
layer_to_color_map = {i:sns.color_palette()[i] for i in range(7)}
def plot_slice(spatial,clusters, figsize=None, color_map=layer_to_color_map, ax=None, s=100):
    (min_x,min_y),(max_x,max_y) = spatial.min(axis=0),spatial.max(axis=0)
    len_x,len_y=max_x-min_x,max_y-min_y
    if not figsize: figsize=(10*(len_x/max(len_x,len_y)),10*(len_y/max(len_x,len_y)))
    if not ax: plt.figure(figsize=figsize)
    colors = [color_map[cluster] for cluster in clusters]
    g = sns.scatterplot(x = spatial[:,0],y =spatial[:,1],linewidth=0,s=s, marker=".",c=colors,ax=ax)
    if not ax: ax=g
    if ax:
        ax.invert_yaxis()
        ax.axis('off')

plot_slice(Z, best_cluster)